
# US Macro Correlation Study  
### Inflation, Unemployment & Interest Rates (2020–2026)

This notebook studies the relationship between key US macroeconomic variables:
- Inflation (CPI, monthly % change)
- Unemployment rate
- Effective Federal Funds Rate (EFFR)

The goal is to build a clean monthly dataset and analyze correlations between macro variables.


In [4]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## 1. Load macroeconomic data

The data is sourced from the Federal Reserve Economic Data (FRED) database and stored as CSV files.


In [6]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("data")

cpi = pd.read_csv(DATA_DIR / "CPIAUCSL.csv", parse_dates=["observation_date"])
unrate = pd.read_csv(DATA_DIR / "UNRATE.csv", parse_dates=["observation_date"])
rate = pd.read_csv(DATA_DIR / "EFFR.csv", parse_dates=["observation_date"])


FileNotFoundError: [Errno 2] No such file or directory: 'data/CPIAUCSL.csv'

## 2. Filter the sample period

We restrict the analysis to the period 2020–2026 to focus on the post-COVID macro environment.



start, end = "2020-01-01", "2026-12-31"
cpi = cpi[(cpi["observation_date"] >= start) & (cpi["observation_date"] <= end)]
unrate = unrate[(unrate["observation_date"] >= start) & (unrate["observation_date"] <= end)]
rate = rate[(rate["observation_date"] >= start) & (rate["observation_date"] <= end)]


## 3. Inflation computation

Inflation is computed as the **monthly percentage change** in CPI (MoM).
This captures short-term inflation dynamics.


In [7]:
cpi = cpi.sort_values("observation_date")
cpi["Inflation_MoM"] = cpi["CPIAUCSL"].pct_change() * 100


NameError: name 'cpi' is not defined

## 4. Interest rate aggregation

EFFR is available at a daily frequency.
We convert it to a **monthly average** to match CPI and unemployment data.


rate["EFFR"] = pd.to_numeric(rate["EFFR"], errors="coerce")
rate = rate.sort_values("observation_date")

rate_monthly = (
    rate.set_index("observation_date")["EFFR"]
        .resample("M")
        .mean()
        .rename("EFFR_Monthly")
        .reset_index()
)


## 5. Data alignment and merge

All series are aligned on a common **monthly timeline** using a `month` key.
This avoids mismatches due to different date conventions.


In [8]:
cpi["month"] = cpi["observation_date"].dt.to_period("M")
unrate["month"] = unrate["observation_date"].dt.to_period("M")
rate_monthly["month"] = rate_monthly["observation_date"].dt.to_period("M")

Data = (
    cpi.merge(unrate, on="month", how="inner")
       .merge(rate_monthly[["month", "EFFR_Monthly"]], on="month", how="inner")
)

Data = Data.rename(columns={
    "CPIAUCSL": "CPI",
    "UNRATE": "Unemployment_Rate"
}).sort_values("month")

Data.head()


NameError: name 'cpi' is not defined

## 6. Correlation analysis

We compute simple Pearson correlations between:
- Inflation and unemployment
- Inflation and interest rates
- Unemployment and interest rates


In [9]:
tmp = Data[["Inflation_MoM", "Unemployment_Rate", "EFFR_Monthly"]].dropna()

tmp.corr()


NameError: name 'Data' is not defined

## 7. Time series visualization


In [10]:
plt.figure(figsize=(12, 6))
plt.plot(Data["month"].dt.to_timestamp(), Data["Inflation_MoM"], label="Inflation MoM")
plt.plot(Data["month"].dt.to_timestamp(), Data["Unemployment_Rate"], label="Unemployment Rate")
plt.plot(Data["month"].dt.to_timestamp(), Data["EFFR_Monthly"], label="EFFR Monthly")

plt.title("US Inflation, Unemployment and Interest Rates")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid()
plt.show()


NameError: name 'Data' is not defined

<Figure size 1200x600 with 0 Axes>

In [11]:
## 8. Correlation heatmap


In [12]:
plt.figure(figsize=(7, 5))
sns.heatmap(tmp.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()


NameError: name 'tmp' is not defined

<Figure size 700x500 with 0 Axes>

## Conclusion

This notebook provides a clean and structured way to analyze the relationship between
inflation, unemployment, and interest rates in the US.

Key takeaways:
- Inflation and interest rates tend to move together during tightening cycles.
- Unemployment is more persistent and reacts with a lag.
- Correlation is descriptive and does not imply causality.

This project focuses on **macro intuition and data handling**, not prediction.
